# Hands-on Lab: Interactive Visual Analytics with Folium lab

## Launch Sites Locations Analysis with Folium

In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium DivIcon plugin
from folium.features import DivIcon


### Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

#### TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

In [6]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
folium.Circle(nasa_coordinate, radius=1000, color='#0000FF', fill=True).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)
features = folium.map.FeatureGroup()
for lat, lon, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df["Launch Site"]):
    features.add_child(
        folium.Circle(
            [lat, lon],
            radius=1000,
            color="red",
            fill=True,
        ),
    ),
    features.add_child(
        folium.Marker(
          [lat, lon], popup=label
        )
    )


site_map.add_child(features)

### Task 2: Mark the success/failed launches for each site on the map

In [7]:
marker_cluster = MarkerCluster()

In [8]:
def marker_color(label):
    if label == 1:
        return "blue"
    else:
        return "red"

In [9]:
spacex_df['marker_color'] = spacex_df['class'].apply(marker_color)

#### TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

In [10]:

for index, record in spacex_df.iterrows():

    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  
        icon=folium.Icon(color='white', icon_color=record['marker_color']) 
    )

    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

site_map